# Dataset creation

Run this notebook if you want to create your own dataset

In [ ]:
import numpy as np
import cv2
import numpy as np
import os
import mediapipe as mp
import time
from tqdm import tqdm

In [ ]:
# Name of the folder (Masked, Unmasked, Half_Masked)
name = "Masked"

# Number of pictures to take
datasetSize = 25

if not os.path.exists('Output/faces/' + name):
    os.mkdir('Output/faces/' + name)

# Initialize mediapipe
mp_face_detection = mp.solutions.face_detection

stop = False
count = len(os.listdir('Output/faces/' + name))
datasetSize += count

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    
    # define a video capture object
    vid = cv2.VideoCapture(0)
    
    i = 0
    while(not stop):
        # Capture the video frame
        # by frame
        ret, img = vid.read()
    
        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.flip(img, 1, img)
        
        # Detect faces
        results = face_detection.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        bounded_img = img.copy()

        try:
            if results.detections:
                # Draw the face detection annotations on the image.
                for detection in results.detections:
                    
                    # Get the bounding box of the face
                    # and its characteristics
                    bb = detection.location_data.relative_bounding_box
                    x = bb.xmin
                    y = bb.ymin
                    h = bb.height
                    w = bb.width

                    camWidth = img.shape[1]
                    camHeight = img.shape[0]

                    # Convert x, y, h and w from percentages to pixels
                    x = int(x * camWidth)
                    y = int(y * camHeight)
                    h = int(h * camHeight)
                    w = int(w * camWidth)
                    
                    # Allows some time to move between the captures
                    # Change the modulo value if you want more or less time between captures
                    i = (i + 1) % 35
                    if i == 0:
                        file_name = "face_" + str(count) + ".jpg"
                        
                        # Resize the image to 250x250px
                        f = cv2.resize(img[y:y+h, x:x+w], (250, 250), interpolation = cv2.INTER_AREA)
                        
                        # Save the capture
                        cv2.imwrite('./Output/faces/' + name + '/' + file_name, f)
                        count += 1
                        
                        # Stop if we reached the wanted dataset size
                        if count >= datasetSize:
                            stop = True
                    
                    # Draw a rectangle to represent the bounding box
                    cv2.rectangle(bounded_img, (x, y),(x + w, y + h), (255, 0, 0), 2)
        except:
            pass

        # Increase output size for ergonomy
        bounded_img = cv2.resize(bounded_img, (0,0), fx=1.5, fy=1.5)
        
        # Display the resulting frame
        cv2.imshow('frame', bounded_img)
        
        # Show the remaining frames to capture before the end
        cv2.setWindowTitle('frame', 'frame : ' + str(abs(count - datasetSize)))
        
        # The 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()